In [80]:
# pip install ultralytics

In [81]:
import os
import pandas as pd
import shutil
from PIL import Image
from sklearn.model_selection import train_test_split
import yaml
from ultralytics import YOLO

# Convert CSV Annotations to YOLO Format

In [82]:
images_dir = '/content/drive/MyDrive/AVOS_Images'
csv_file = '/content/drive/MyDrive/open_surgery_bboxes_Jan16.csv'

labels_dir = 'AVOS_labels'  # temporary dir for YOLO-format labels
if not os.path.exists(labels_dir):
    os.makedirs(labels_dir)

In [83]:
df = pd.read_csv(csv_file)
print("CSV Columns:", df.columns.tolist())

CSV Columns: ['image_name', 'x1', 'y1', 'x2', 'y2', 'class']


In [84]:
df.head()

,image_name,x1,y1,x2,y2,class
0,-3gFrKiC99I-000002399.jpg,1.0,477.0,357.0,719.0,hand
1,-3gFrKiC99I-000002399.jpg,166.0,1.0,943.0,716.0,hand
2,-3gFrKiC99I-000002399.jpg,682.0,1.0,1278.0,439.0,hand
3,-3gFrKiC99I-000003199.jpg,242.0,1.0,478.0,383.0,forceps
4,-3gFrKiC99I-000003199.jpg,1.0,405.0,565.0,710.0,hand


In [85]:
df['class'].unique()

array(['hand', 'forceps', 'bovie', 'needledriver', nan], dtype=object)

deleting images that doesn't belong to the surgery

In [86]:
# images_to_delete = []
# grouped = df.groupby('image_name')

# for image_name, group in grouped:
#     if group['class'].isna().all():
#         images_to_delete.append(image_name)

# print("Images to be deleted:", images_to_delete)
# print(len(images_to_delete))

In [87]:
# images_folder = "/content/drive/MyDrive/AVOS_Images"

# for image_name in images_to_delete:
#     image_path = os.path.join(images_folder, f"{image_name}")
#     if os.path.exists(image_path):
#         os.remove(image_path)
#         print(f"Deleted {image_path}")
#     else:
#         print(f"Image {image_path} not found!")

In [88]:
df = df.dropna(how='any')
df['class'].unique()

array(['hand', 'forceps', 'bovie', 'needledriver'], dtype=object)

In [89]:
df = df[df['image_name'] != 'ILZA2bFVGPE-000004967.jpg']

In [90]:
# Create a mapping from class names to indices (sorted alphabetically)
classes = sorted(df['class'].unique())
class_to_idx = {c: i for i, c in enumerate(classes)}
print("Class mapping:", class_to_idx)

grouped = df.groupby('image_name')

for image_name, group in grouped:
    image_filename = f"{image_name}"
    image_path = os.path.join(images_dir, image_filename)

    if not os.path.exists(image_path):
        print(f"Warning: Image file {image_path} not found. Skipping.")
        continue

    with Image.open(image_path) as img:
        width, height = img.size

    # Prepare YOLO annotation lines (one per bounding box)
    lines = []
    for _, row in group.iterrows():
        xmin = row['x1']
        ymin = row['y1']
        xmax = row['x2']
        ymax = row['y2']

        # Calculate normalized coordinates (center_x, center_y, width, height)
        x_center = ((xmin + xmax) / 2) / width
        y_center = ((ymin + ymax) / 2) / height
        w_norm = (xmax - xmin) / width
        h_norm = (ymax - ymin) / height

        cls = row['class']
        cls_idx = class_to_idx[cls]

        # Format: class_index x_center y_center width height
        lines.append(f"{cls_idx} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}")

    # Write annotation to a text file with the same base name as the image
    label_filename = f"{image_name}.txt"
    label_path = os.path.join(labels_dir, label_filename)
    with open(label_path, 'w') as f:
        f.write("\n".join(lines))

print("Conversion to YOLO format completed.")

Class mapping: {'bovie': 0, 'forceps': 1, 'hand': 2, 'needledriver': 3}
Conversion to YOLO format completed.


In [91]:
for filename in os.listdir(labels_dir):
    if filename.endswith('.jpg.txt'):
        new_filename = filename.replace('.jpg.txt', '.txt')
        old_path = os.path.join(labels_dir, filename)
        new_path = os.path.join(labels_dir, new_filename)
        os.rename(old_path, new_path)

In [92]:
image_names = list(grouped.groups.keys())
print(image_names)

['-3gFrKiC99I-000002399.jpg', '-3gFrKiC99I-000003199.jpg', '-3gFrKiC99I-000003999.jpg', '-3gFrKiC99I-000004799.jpg', '-3gFrKiC99I-000005599.jpg', '-3gFrKiC99I-000006399.jpg', '-3gFrKiC99I-000007199.jpg', '-3gFrKiC99I-000007999.jpg', '-3gFrKiC99I-000008799.jpg', '-3gFrKiC99I-000009599.jpg', '-DGQsmwXmLg-000000653.jpg', '-DGQsmwXmLg-000000871.jpg', '-DGQsmwXmLg-000001089.jpg', '-DGQsmwXmLg-000001307.jpg', '-DGQsmwXmLg-000001525.jpg', '-DGQsmwXmLg-000001743.jpg', '-DGQsmwXmLg-000001961.jpg', '-DGQsmwXmLg-000002179.jpg', '-DGQsmwXmLg-000002397.jpg', '-DGQsmwXmLg-000002615.jpg', '0EeZIRDKYO4-000000551.jpg', '0EeZIRDKYO4-000000689.jpg', '0EeZIRDKYO4-000000827.jpg', '0EeZIRDKYO4-000000965.jpg', '0EeZIRDKYO4-000001103.jpg', '0EeZIRDKYO4-000001241.jpg', '0EeZIRDKYO4-000001379.jpg', '0EeZIRDKYO4-000001517.jpg', '0EeZIRDKYO4-000001655.jpg', '0VMxXsqmxYg-000000272.jpg', '0VMxXsqmxYg-000000363.jpg', '0VMxXsqmxYg-000000454.jpg', '0VMxXsqmxYg-000000545.jpg', '0VMxXsqmxYg-000000636.jpg', '0VMxXsqmxYg-

In [93]:
image_names = [name[:-4] if name.endswith(".jpg") else name for name in image_names]
print(image_names)

['-3gFrKiC99I-000002399', '-3gFrKiC99I-000003199', '-3gFrKiC99I-000003999', '-3gFrKiC99I-000004799', '-3gFrKiC99I-000005599', '-3gFrKiC99I-000006399', '-3gFrKiC99I-000007199', '-3gFrKiC99I-000007999', '-3gFrKiC99I-000008799', '-3gFrKiC99I-000009599', '-DGQsmwXmLg-000000653', '-DGQsmwXmLg-000000871', '-DGQsmwXmLg-000001089', '-DGQsmwXmLg-000001307', '-DGQsmwXmLg-000001525', '-DGQsmwXmLg-000001743', '-DGQsmwXmLg-000001961', '-DGQsmwXmLg-000002179', '-DGQsmwXmLg-000002397', '-DGQsmwXmLg-000002615', '0EeZIRDKYO4-000000551', '0EeZIRDKYO4-000000689', '0EeZIRDKYO4-000000827', '0EeZIRDKYO4-000000965', '0EeZIRDKYO4-000001103', '0EeZIRDKYO4-000001241', '0EeZIRDKYO4-000001379', '0EeZIRDKYO4-000001517', '0EeZIRDKYO4-000001655', '0VMxXsqmxYg-000000272', '0VMxXsqmxYg-000000363', '0VMxXsqmxYg-000000454', '0VMxXsqmxYg-000000545', '0VMxXsqmxYg-000000636', '0VMxXsqmxYg-000000726', '0VMxXsqmxYg-000000817', '0VMxXsqmxYg-000000908', '0VMxXsqmxYg-000000999', '0VMxXsqmxYg-000001090', '0oxhRpSq850-000000905',

# Split Dataset into Training and Validation Sets

In [94]:
train_ids, val_ids = train_test_split(image_names, test_size=0.2, random_state=42)
print(f"Total images: {len(image_names)} | Train: {len(train_ids)} | Val: {len(val_ids)}")

train_images_dir = '/content/AVOS_train_images'
val_images_dir = '/content/AVOS_val_images'
train_labels_dir = '/content/AVOS_train_labels'
val_labels_dir = '/content/AVOS_val_labels'

Total images: 2086 | Train: 1668 | Val: 418


In [95]:
for d in [train_images_dir, val_images_dir, train_labels_dir, val_labels_dir]:
    if not os.path.exists(d):
        os.makedirs(d)

def copy_files(image_names, dest_images_dir, dest_labels_dir):
    for image_name in image_names:
        image_file = os.path.join(images_dir, f"{image_name}.jpg")
        label_file = os.path.join(labels_dir, f"{image_name}.txt")
        if os.path.exists(image_file):
            shutil.copy(image_file, dest_images_dir)
        else:
            print(f"Image file {image_file} not found!")
        if os.path.exists(label_file):
            shutil.copy(label_file, dest_labels_dir)
        else:
            print(f"Label file {label_file} not found!")

copy_files(train_ids, train_images_dir, train_labels_dir)
copy_files(val_ids, val_images_dir, val_labels_dir)
print("Dataset split into training and validation sets.")

Dataset split into training and validation sets.


In [96]:
for filename in os.listdir(val_images_dir):
    if filename.endswith('.jpg'):
        base_name = os.path.splitext(filename)[0]
        label_filename = f"{base_name}.txt"
        src_label = os.path.join(val_labels_dir, label_filename)
        dst_label = os.path.join(val_images_dir, label_filename)

        if os.path.exists(src_label) and not os.path.exists(dst_label):
            shutil.copy(src_label, dst_label)
            print(f"Copied {src_label} to {dst_label}")
        elif not os.path.exists(src_label):
            print(f"Label file {src_label} not found!")

Copied /content/AVOS_val_labels/_lCgV4ZgAao-000004973.txt to /content/AVOS_val_images/_lCgV4ZgAao-000004973.txt
Copied /content/AVOS_val_labels/XZleSz21hik-000003842.txt to /content/AVOS_val_images/XZleSz21hik-000003842.txt
Copied /content/AVOS_val_labels/ftkqy5JBuSM-000003539.txt to /content/AVOS_val_images/ftkqy5JBuSM-000003539.txt
Copied /content/AVOS_val_labels/PsYnEXGxf-M-000009525.txt to /content/AVOS_val_images/PsYnEXGxf-M-000009525.txt
Copied /content/AVOS_val_labels/_35HaYVPBIA-000000365.txt to /content/AVOS_val_images/_35HaYVPBIA-000000365.txt
Copied /content/AVOS_val_labels/nwqyKvQ_mNk-000005158.txt to /content/AVOS_val_images/nwqyKvQ_mNk-000005158.txt
Copied /content/AVOS_val_labels/Z4Z03ebdSa8-000006179.txt to /content/AVOS_val_images/Z4Z03ebdSa8-000006179.txt
Copied /content/AVOS_val_labels/MQ0w6hXABDc-000002799.txt to /content/AVOS_val_images/MQ0w6hXABDc-000002799.txt
Copied /content/AVOS_val_labels/qaAoYqg2vWo-000007856.txt to /content/AVOS_val_images/qaAoYqg2vWo-000007

In [97]:
# for filename in os.listdir(train_labels_dir):
#      if filename.endswith('.jpg.txt'):
#         new_filename = filename.replace('.jpg.txt', '.txt')
#         old_path = os.path.join(labels_dir, filename)
#         new_path = os.path.join(labels_dir, new_filename)
#         os.rename(old_path, new_path)

# for filename in os.listdir(val_labels_dir):
#      if filename.endswith('.jpg.txt'):
#         new_filename = filename.replace('.jpg.txt', '.txt')
#         old_path = os.path.join(labels_dir, filename)
#         new_path = os.path.join(labels_dir, new_filename)
#         os.rename(old_path, new_path)

#  Generate Dataset YAML File for YOLOv8

In [98]:
dataset_yaml = {
    'train': os.path.abspath(train_images_dir),
    'val': os.path.abspath(val_images_dir),
    'nc': len(classes),
    'names': classes
}

yaml_file = 'surgery.yaml'
with open(yaml_file, 'w') as f:
    yaml.dump(dataset_yaml, f, default_flow_style=False)

print(f"Dataset YAML file created: {yaml_file}")
print(dataset_yaml)

Dataset YAML file created: surgery.yaml
{'train': '/content/AVOS_train_images', 'val': '/content/AVOS_val_images', 'nc': 4, 'names': ['bovie', 'forceps', 'hand', 'needledriver']}


In [99]:
val_images_dir = "/content/AVOS_val_images"
val_labels_dir = "/content/AVOS_val_labels"

image_files = [f for f in os.listdir(val_images_dir) if f.endswith('.jpg')]
missing_labels = []

for img_file in image_files:
    label_file = os.path.splitext(img_file)[0] + ".txt"
    if not os.path.exists(os.path.join(val_labels_dir, label_file)):
        missing_labels.append(label_file)

if missing_labels:
    print("Missing label files for:", missing_labels)
else:
    print("All validation images have label files!")

All validation images have label files!


# Train YOLOv8 Model on Surgical Dataset

In [100]:
model = YOLO('yolov8n.pt')
results = model.val(data='surgery.yaml')

# Train the model on your dataset; adjust hyperparameters as needed.
model.train(data='surgery.yaml', epochs=20, imgsz=640)

Ultralytics 8.3.95 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs


val: Scanning /content/AVOS_val_images... 613 images, 0 backgrounds, 0 corrupt: 100%|██████████| 613/613 [00:00<00:00, 2120.45it/s]


val: New cache created: /content/AVOS_val_images.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 39/39 [00:07<00:00,  5.17it/s]


                   all        613       1869     0.0227      0.141     0.0137    0.00619
                person        153        154    0.00401      0.422    0.00578    0.00175
               bicycle        120        145    0.00617     0.0138    0.00324    0.00114
                   car        565       1277     0.0357     0.0728      0.021     0.0114
            motorcycle        199        293     0.0448     0.0546     0.0249     0.0105
Speed: 0.2ms preprocess, 2.9ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/val10
Ultralytics 8.3.95 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=surgery.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train13, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=F

...

                   all        613       1869      0.025   0.000392     0.0136    0.00285



20 epochs completed in 0.218 hours.
Optimizer stripped from runs/detect/train13/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train13/weights/best.pt, 6.2MB

Validating runs/detect/train13/weights/best.pt...
Ultralytics 8.3.95 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:04<00:00,  4.65it/s]


                   all        613       1869     0.0625   0.000196     0.0322    0.00966
                 bovie        153        154          0          0          0          0
               forceps        120        145          0          0          0          0
                  hand        565       1277       0.25   0.000783      0.129     0.0387
          needledriver        199        293          0          0          0          0
Speed: 0.2ms preprocess, 2.0ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/train13


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7eafe4750350>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

In [101]:
model.save("surgery_model.pt")